In [23]:
"""
Original linear model: y = b + wx (Too simple to use)
Sigmoid function model: Suppose we have three sigmoid function

"""
# Viewers for previous three days: X = [25, 20, 21]
# There are weight for no. of sigmoid and no. of features. So Wij represents weight for xj for i-th sigmoid
# Bias: different bias for different functions

"""
r1 = b1 + w11x1 + w12x2 + w13x3
r2 = b2 + w21x1 + w22x2 + w23x3
r3 = b3 + w31x1 + w32x2 + w33x3

Where 
result vector: R = [r1, r2, r3]
weight matrix W = [[w11, w12, w13], [w21, w22, w23], [w31, w32, w33]]
input vector X = [x1, x2, x3]
bias vector B = [b1, b2, b3]
"""

import numpy as np

# Define the sigmoid function
def sigmoid(r):
    return 1 / (1 + np.exp(-r))

# Set a random seed for reproducibility
np.random.seed(42)

# Define the input vector
original_x = np.array([33, 28, 32])

def model(X, layers):
    # Randomly generate a 3x3 weight matrix with values between -1 and 1
    W = 2 * np.random.rand(3, 3) - 1

    # Randomly generate a 3-element bias vector with values between 50 and 60
    B = 50 + 10 * np.random.rand(3)

    # Compute the result
    R = np.dot(W, X) + B

    # R = W * X + B
    print(R)

    # Apply the sigmoid function to each element of R
    sigmoid_R = sigmoid(R)

    print("Result after applying sigmoid:", sigmoid_R)

    # Define the coefficient vector (c1, c2, c3)
    C = np.random.rand(3) * 100 + 0.5

    # C * sigmoid_R
    sigmoid_R_scaled = C * sigmoid_R
    print("sigmoid_R_scaled is:", sigmoid_R_scaled)

    # Randomly generate a 3-element bias vector with values between 0 and 0.1
    B2 = 0.1 * np.random.rand(3)

    print("Randomly generated bias vector (B2):")
    print(B2)

    # B2 + sigmoid_R_scaled
    Y = B2 + sigmoid_R_scaled
    
    if layers > 0:
        print("layers is ->", layers)
        print("Y is ->", Y)
        layers -= 1
        Y = model(Y, layers)
    return Y

def deep_learn(original_data, layers):
    return model(original_data, layers)

print("Final Result is :", deep_learn(original_x, 3))

[88.88798706 15.43799807 57.50984184]
Result after applying sigmoid: [1.        0.9999998 1.       ]
sigmoid_R_scaled is: [83.74426408 21.73390678 18.68249672]
Randomly generated bias vector (B2):
[0.01834045 0.03042422 0.05247564]
layers is -> 3
Y is -> [83.76260453 21.764331   18.73497236]
[ 38.84502499 -18.52480036  44.2652403 ]
Result after applying sigmoid: [1.00000000e+00 9.01117496e-09 1.00000000e+00]
sigmoid_R_scaled is: [6.12544852e+01 1.58167859e-07 7.00515930e+00]
Randomly generated bias vector (B2):
[0.09488855 0.0965632  0.08083973]
layers is -> 2
Y is -> [61.34937374  0.09656336  7.08599903]
[35.18482687 45.63254936 -5.26878399]
Result after applying sigmoid: [1.         1.         0.00512348]
sigmoid_R_scaled is: [55.17102793 18.98544555  0.49932685]
Randomly generated bias vector (B2):
[0.07751328 0.09394989 0.08948274]
layers is -> 1
Y is -> [55.24854122 19.07939545  0.58880959]
[79.99880689  1.65706725 34.78820925]
Result after applying sigmoid: [1.         0.83984392